Library

In [1]:
from pyvi import ViTokenizer, ViPosTagger
import numpy as np
import pandas as pd
import gensim
import sklearn
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.layers import *
from keras.layers import *
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from keras.utils import to_categorical

ModuleNotFoundError: No module named 'pyvi'

Data Preparation

In [ ]:
datanewscontent=pd.read_excel(r'path_to_newscontent.xlsx','Sheet1')
datacomment=pd.read_excel(r'path_to_comments.xlsx','Sheet1')

In [ ]:
datacmt=[]
for d in datacomment['comment']:
    e=ViTokenizer.tokenize(str(d))
    datacmt.append(e)
labelcmt=datacomment['label']

In [ ]:
datanews=[]
for d in datanewscontent['all_lower']:
    e=ViTokenizer.tokenize(str(d))
    datanews.append(e)
labelnews=datanewscontent['label']

In [ ]:
def truncatedvectors(data,n_components=300):
  svd_ngram = TruncatedSVD(n_components=n_components, random_state=42)
  svd_ngram.fit(data)
  return svd_ngram.transform(data)

Word2Vec

In [ ]:
from gensim.models import Word2Vec,KeyedVectors 
import os
word2vec_model_path ="path_to_wikivimodel.bin"
model = KeyedVectors.load_word2vec_format(word2vec_model_path,binary=True, unicode_errors='ignore')
vocab = model.wv.vocab
wv = model.wv

In [ ]:
def get_word2vec_data(X):
    word2vec_data = []
    for x in X:
        sentence = []
        for word in x.split(" "):
            if word in vocab:
              sentence=sentence+wv[word].ravel().tolist()
        word2vec_data.append(sentence)

    return word2vec_data
def change_to_word2vec(data):
  data2vec=get_word2vec_data(data)
  lengthOfdata=[len(data2vec[i]) for i,n in enumerate(data2vec)]
  for i,n in enumerate(data):
    if(len(data2vec[i])<max(lengthOfdata)):
      for j in range(1,(max(lengthOfdata)-len(data2vec[i]))+1):
        data2vec[i].append(0)
  return truncatedvectors(np.array(data2vec))

In [ ]:
X_data_w2v_cmt=change_to_word2vec(datacmt)
X_data_w2v_news=change_to_word2vec(datanews)

TF-IDF

In [ ]:
def tfidf(data):
  tfidf_vect_ngram = TfidfVectorizer(analyzer='word', max_features=30000, ngram_range=(1, 2))
  tfidf_vect_ngram.fit(data)
  X_data_tfidf_ngram =  tfidf_vect_ngram.transform(data)
  return truncatedvectors(X_data_tfidf_ngram)

In [ ]:
X_data_tfidf_cmt=tfidf(datacmt)
X_data_tfidf_news=tfidf(datanews)

Bag of words

In [ ]:
def bow(data):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(dataall)
  datacmtbow = tokenizer.texts_to_sequences(data)
  datacmtbow= pad_sequences(datacmtbow, maxlen=300)
  return datacmtbow

In [ ]:
X_data_bow_cmt=bow(datacmt)
X_data_bow_news=bow(datanews)

LSTM Model

In [ ]:
#Change: [X_data_tfidf_cmt,  X_data_w2v_cmt, X_data_bow_cmt],labelcmt , [X_data_tfidf_news,X_data_bow_news,X_data_w2v_news], labelnews
X_train, X_val, y_train, y_val = train_test_split(X_data_tfidf_cmt, labelcmt, test_size=0.2, random_state=42)

In [ ]:
#learning_rate: [1e-5,5e-5,1e-4,5e-4,1e-3,5e-3]
opt = Adam(lr=0.001)
def create_lstm_model():
    input_layer = Input(shape=(300,))
    layer = Reshape((10, 30,))(input_layer)
    layer = LSTM(128, activation='relu',dropout=0.5, recurrent_dropout=0.5)(layer)
    layer = Dense(64, activation='relu')(layer)
    layer = Dense(32, activation='relu')(layer)
    output_layer = Dense(3, activation='softmax')(layer)
    classifier = Model(input_layer, output_layer)
    classifier.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return classifier

In [ ]:
y_train_lstm_encode = to_categorical(y_train_lstm)
y_val_lstm_encode=to_categorical(y_val_lstm)
classifier = create_lstm_model()
classifier.fit(x_train_lstm, y_train_lstm_encode, validation_data=(x_val_lstm, y_val_lstm_encode), epochs=200, batch_size=32)
#change different epoch and batch_size

In [ ]:
y_pred = classifier.predict(x_val_lstm, batch_size=32, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
print(classification_report(y_val_lstm, y_pred_bool))